## Non linear Optimization

State estimation in SLAM depends on the following two equations:

- $x_k = f(x_{k-1}, u_k) + w_k$
- $z_{k,j} = h(y_j, x_k) + v_{k,j}$

The $x_k$ is the camera pose in 6d, $z_{k,j}$ is the image position of the observation $y_j$.
The $u_k$ is the input data at time $k$. 
The input data can be a set of 2d points that are known to belong to the same object in a sequence of images.
It can be a mixed set of 3D and 2D points, or a set of 3D points (see Camera-Motion notebook for the related discussion).  

The $x_k$ can be expressed as $R_k y_j + t_k$. 
Under this expression the $x_k$ and $z_{k,j}$ is bound with the following equation:
$$s z_{k,j} = K(R_k y_j + t_k) = KT y_j$$
where $T \in SE3D$, $s$ represents the distance of pixels, and $K$ represents the intrinsic camera matrix.

The $w_k$ and $v_{k,j}$ are noise terms. They are usually assume to be gaussian with 0 mean.

Given the presence of noise, our problem can be formulated as a conditional probability distribution like $P(x, y | z, u)$ meaning that given the pixel position $z$ and input $u$ what is the probability of pose being $x$ and observation being $y$. 

The values that maximize this probability distribution minimize the error and noise in the system.
From the bayes rule, $argmax P(x, y | z, u) = argmax P(z, u | x, y) * P(x, y)$, since we may not know the prior $P(x,y)$ we can also ignore that and transform the Maximizing posterior distribution problem to Maximium likelihood estimation problem, and end up with $$argmax P(x, y| z, u) = argmax P(z, u | x, y)$$.

This means that we need to minimize the following terms:
- $e_{u, k} = x_k - f(x_{k-1}, u_k)$
- $e_{z, j, k} = z_{k, j} - h(x_k, y_j)$

## Non linear least-square

The term non linear means that our function do not have a linear form like a straight line or a parabola. In the linear case the derivative is constant.
In the non linear case, like in exponential function or logarithmic function, the derivative is not constant.

Let there be a $m$ set of points and observations such that $(x_1, y_1), (x_2, y_2), \dots, (x_m, y_m)$
Let $h$ be the hypothesis function that binds $x_i$ to $y_i$ using the parameter vector $\alpha$ such that $$y_i = h(x_i, \alpha)$$ where $\alpha \in \mathbb{R}^n$

Let $r$ be the residual function, that measures the difference between the hypothesis function and the observation $$r(x_i, y_i,\alpha) = y_i - h(x_i, \alpha)$$

Let $F$ be the objective function that measures the total error resulting from the parameters through summation of squared residuals:
$$F(x, y, \alpha) = \sum_{i=1}^m r(x_i, y_i, \alpha)^2$$

Since we can't change the data and the observation, our ultimiate goal is to find the set of parameters, $\alpha$, that minimizes the objective function. The minimum/maximum value of the objective function comes from $\frac{\partial F(x, y, \alpha)}{\alpha} = 0$. 

Now we know that minimum of a function that sums bunch of values is greater or equal to minimum of functions that is being summed.

Formally, $\forall p, f(p) \ge min(f(p))$ then $\forall p, f(p) + f(p) \ge min(f(p)) + min(f(p))$ and consequently $\forall p, min(f(p) + f(p)) \ge min(f(p) + f(p))$

Consequently the minimum of $min(F(x, y, \alpha)) \ge \sum_{i=1}^m min(r(x_i, y_i, \alpha)^2)$. 
Hence, we can get away with only finding the minimum of $r(\dots)^2$.

Let's take a closer look at $r$ then.

Here just like the objective function $F$, we'll use the derivative based on $\alpha$ as in $\frac{\partial k(\alpha)}{\alpha}$ where $k(\alpha) = r(x_i, y_i, \alpha)^2$ and $k(\alpha) \in \mathbb{R}$

Let $\alpha = \alpha^s + \nabla \alpha$ where $\alpha^s$ the state of a value provided to $\alpha$ during the beginning of approximation at the $s$th iteration and $\nabla \alpha$ represent a shift vector that helps to approximate $\alpha$ with $\alpha^s$. 

From here it should be evident that $\alpha^s$ approximates $\alpha$ better and better as the $\nabla \alpha$ gets smaller, so in practice when $\nabla \alpha$, gets smaller than a threshold, we stop iterating.

The shift vector plays its part when we expand the $k(\alpha)$ using the taylor series:
$$k(\alpha^s + \nabla \alpha) \approx k(\alpha^s) + k'(\alpha^s)\nabla \alpha + \frac{k''(\alpha^s)(\nabla \alpha)^2}{2!}$$

Here we are expanding until the second order.

Now, the idea is that the minimum of this approximation should be the minimum of the $k$, so we try to find the minimum value of this approximation. 
The method is similar to above. We set derivative of $k$ to 0.
In this case, since the value $\alpha^s$ is provided during the iteration, we compute the derivative against $\nabla \alpha$:

$$\frac{\partial k(\alpha^s + \nabla \alpha)}{\nabla \alpha} = 0 =
0 + k'(\alpha^s)+ \frac{2 * k''(\alpha^s) \nabla \alpha}{2}$$

The equation simplifies as:
$$0 = k'(\alpha^s) + k''(\alpha^s) \nabla \alpha$$
and consequently:
$$-k'(\alpha^s) = k''(\alpha^s) \nabla \alpha$$
The $-k'(\alpha^s)$ is the gradient of $k$ evaluated at $\alpha^s$.
The $k''(\alpha^s)$ is the hessian matrix of $k$ evaluated at $\alpha^s$.
Hence the final form of the equation:
$$-g(\alpha^s) = H(\alpha^s) \nabla \alpha$$

This is classic $Ax=b$ type equation and can be solved using decomposition strategies.

The $g(\alpha^s) \in \mathbb{R}^n$ is a vector:
$$g_k(\alpha^s) = [\frac{\partial k}{\alpha^s_1}, \dots, \frac{\partial k}{\alpha^s_n}]$$

The $H(\alpha^s) \in \mathbb{R}^{n \times n}$ is a matrix based on the gradient:
$$H_k(\alpha^s) = \begin{pmatrix}
\frac{\partial^2 k}{\partial (\alpha^s_1)_1} & \frac{\partial^2 k}{\partial (\alpha^s_1)_2} & \dots & \frac{\partial^2 k}{\partial (\alpha^s_1)_n} \\
\frac{\partial^2 k}{\partial (\alpha^s_2)_1} & \frac{\partial^2 k}{\partial (\alpha^s_2)_2} & \dots & \frac{\partial^2 k}{\partial (\alpha^s_2)_n} \\
\dots \\
\frac{\partial^2 k}{\partial (\alpha^s_n)_1} & \frac{\partial^2 k}{\partial (\alpha^s_n)_2} & \dots & \frac{\partial^2 k}{\partial (\alpha^s_n)_n}
\end{pmatrix}$$

The problem is computing the $H$ takes a lot of time, so in reality one would approximate it using the jacobian like: $$J(\alpha^s)J^T(\alpha^s) \nabla \alpha = -g(\alpha^s) \nabla \alpha$$
This is called the normal equation or *Gauss-Newton* equation.

Its incomplete derivation is given below

## Deriving Gauss-Newton equation

Consider $m$ data points: 
$(x_1, y_1), (x_2, y_2), (x_i, y_i), \dots (x_m, y_m)$ where $x_i, y_i \in \mathbb{R}^n$

The problem is stated as minimizing $S$ in $S = \sum_{i=1}^m (r_i)^2$ where $r_i = y_i - f(x_i, \beta)$.

The minimum value occurs when gradient is 0.
Here the $x_i$ and $y_i$ are given so the only term we can manipulate for finding the minimum is effectively $\beta$.

Notice that we have three functions $S$, the objective function, $(r_i)$ the residual function and $f$ the model function.
We are interested in the minimum of $S$, so we set its derivative to 0:
$S'(\beta) = 0$. Let's rewrite the objective function as a composition of other functions to better express the derivative through chain rule:
$$S(\beta) = K(r(f(\beta)))$$ where $K(r) = \sum_{i=1}^m r^2$, $r(\hat{y}_i) = y_i - \hat{y}_i$ and $\hat{y}_i = f(x_i, \beta)$

In function composition notation $S(\beta) = (K \circ r \circ f)(\beta)$

The derivative of the composition function would be:
$$S'(\beta) = K'((r \circ f)(\beta)) * (r \circ f)'(\beta)$$

The derivative of second composition $(r \circ f)$ would be:
$$(r \circ f)'(\beta) = r'(f(\beta)) * f'(\beta)$$

The final form of the derivative would be:
$$S'(\beta) = K'((r \circ f)(\beta)) * r'(f(\beta)) * f'(\beta)$$

or in more familiar terms
$$S'(\beta) = K'(r) * r'(\hat{y}) * f'(\beta)$$

Let's plug the terms to back the equation step by step:
$$K'(r) = \nabla (\sum_{i=1}^m r^2) = 2 \sum_{i=1}^m r$$

$$r'(\hat{y}) = \nabla (y_i - \hat{y}) = -1$$ where $\hat{y} = f(\beta)$

$$f'(x, \beta) = \begin{pmatrix}
  \frac{\partial x_1}{\beta_1} & \frac{\partial x_1}{\beta_2} & \dots & \frac{\partial x_1}{\beta_j} & \dots & \frac{\partial x_1}{\beta_n} \\
  \frac{\partial x_2}{\beta_1} & \frac{\partial x_2}{\beta_2} & \dots & \frac{\partial x_2}{\beta_j} & \dots & \frac{\partial x_2}{\beta_n} \\
  \dots \\
  \frac{\partial x_i}{\beta_1} & \frac{\partial x_i}{\beta_2} & \dots & \frac{\partial x_i}{\beta_j} & \dots & \frac{\partial x_i}{\beta_n} \\
  \dots \\
  \frac{\partial x_m}{\beta_1} & \frac{\partial x_m}{\beta_2} & \dots & \frac{\partial x_m}{\beta_j} & \dots & \frac{\partial x_m}{\beta_n}
\end{pmatrix}$$

The model function $f(x, \beta)$ can be expanded using the first order taylor series: $$f(x_i, \beta) \approx f(x_i, \beta^k) + f'(x_i, \beta^k)*(\beta - \beta^k)$$ where $\beta^k$ represents the state of a value provided to $\beta$ during the beginning of approximation at the $k$th iteration.
In an alternative formulation, let $\beta = \beta^k + \nabla \beta$:
$$f(x_i, \beta^k + \nabla \beta) \approx f(x_i, \beta^k) + f'(x_i, \beta^k) * \nabla \beta$$

The $f'(x, \beta^k)$ is essentially 
$$ J(f) = f'(x, \beta^k) = \begin{pmatrix}
  \frac{\partial x_1}{\beta^k_1} & \frac{\partial x_1}{\beta^k_2} & \dots & \frac{\partial x_1}{\beta^k_j} & \dots & \frac{\partial x_1}{\beta^k_n} \\
  \frac{\partial x_2}{\beta^k_1} & \frac{\partial x_2}{\beta^k_2} & \dots & \frac{\partial x_2}{\beta^k_j} & \dots & \frac{\partial x_2}{\beta^k_n} \\
  \dots \\
  \frac{\partial x_i}{\beta^k_1} & \frac{\partial x_i}{\beta^k_2} & \dots & \frac{\partial x_i}{\beta^k_j} & \dots & \frac{\partial x_i}{\beta^k_n} \\
  \dots \\
  \frac{\partial x_m}{\beta^k_1} & \frac{\partial x_m}{\beta^k_2} & \dots & \frac{\partial x_m}{\beta^k_j} & \dots & \frac{\partial x_m}{\beta^k_n}
\end{pmatrix}$$

It is evident that $\beta^k$ approximates $\beta$ better and better as the $\nabla \beta$ gets smaller, ( second term in plus gets smaller so the perturbation gets smaller ), so in practice when $\nabla \beta$, also known as the shift vector, gets smaller than a threshold, we stop iterating.

Now let's rewrite the final form of the derivative after the taylor expansion:
$$S'(\beta) = 2 \sum_{i=1}^m r * -1 * f'(x, \beta^k)$$ where $*$ stands for matrix multiplication. 

We can also substitute the $f(x, \beta^k)$ in $r$ to govern the $y$ residual through $\beta^k$:
$$S'(\beta) = 2 \sum_{i=1}^m (y_i - (f(x_i, \beta^k) + f_i'(x, \beta^k) * \nabla \beta) * -1 * f'(x, \beta^k)$$ where $f'_i(x, \beta^k)$ represents the ith row vector in the jacobian $J(f)$ matrix.

When we distribute the minus sign, we get:
$$S'(\beta) = 2 \sum_{i=1}^m (y_i - f(x_i, \beta^k) - f_i'(x, \beta^k) * \nabla \beta) * -1 * f'(x, \beta^k)$$

Since we are looking for $S'(\beta) = 0$ due to minimization requirement, this equation morphs into:
$$0 = 2 \sum_{i=1}^m (y_i - f(x_i, \beta^k) - f_i'(x, \beta^k) * \nabla \beta) \cdot -1 * f'(x, \beta^k)$$

Now let's move the negative sign to the coefficient:
$$0 = -2 \sum_{i=1}^m (y_i - f(x_i, \beta^k) - f_i'(x, \beta^k) * \nabla \beta) * f'(x, \beta^k)$$

Let $\nabla y_i = y_i - f(x_i, \beta^k)$, then the equation morphs into:
$$0 = -2 \sum_{i=1}^m (\nabla y_i - f_i'(x, \beta^k) * \nabla \beta) * f'(x, \beta^k)$$

Notice that right side of the equation is all multiplications, so we can get rid of the coefficient -2 at this point:
$$0 = \sum_{i=1}^m (\nabla y_i - f_i'(x, \beta^k) * \nabla \beta) * f'(x, \beta^k)$$

We distribute the $f'(x, \beta^k)$ to summation terms:
$$0 = \sum_{i=1}^m \nabla y_i * f'(x, \beta^k) - \sum_{i=1}^m f_i'(x, \beta^k) * \nabla \beta * f'(x, \beta^k))$$

We move the second summation to the left side of the equation and achieve the final form of the equation ?:
$$\sum_{i=1}^m f_i'(x, \beta^k) * \nabla \beta * f'(x, \beta^k) = \sum_{i=1}^m \nabla y_i * f'(x, \beta^k)$$


The term $f_i'(x, \beta^k) * \nabla \beta$ is effectively:
$f_i'(x, \beta^k) * \nabla \beta = \frac{\partial x_i}{\beta^k_1} * \nabla \beta_1 + \dots \frac{\partial x_i}{\beta^k_n} * \nabla \beta_n$

From here one can see that $\nabla \beta$ distributes to summation like:
$$(\frac{\partial x_1}{\beta^k_1} + \dots + \frac{\partial x_m}{\beta^k_1}) \nabla \beta^k_1 + \dots + (\frac{\partial x_1}{\beta^k_n} + \dots + \frac{\partial x_m}{\beta^k_n}) \nabla \beta^k_n$$

This distribution can be abbreviated as the following:
$$\sum_{j=1}^n \sum_{i=1}^{m} J_{ij}\nabla \beta_j$$

The equation becomes:
$$\sum_{j=1}^n \sum_{i=1}^{m} J_{ij}\nabla \beta_j * J(f) = \sum_{i=1}^m \nabla y_i * J(f)$$

We multiply both sides with the inverse of jacobian:
$$\sum_{j=1}^n \sum_{i=1}^{m} J_{ij}\nabla \beta_j * J(f) * J^{-1}(f) = \sum_{i=1}^m \nabla y_i * J(f) * J^{-1}(f)$$

Since matrix multiplication has associative property when dimensions match, we can do $J(f) * J^{-1} = I$ and substitute it to the equation:
$$\sum_{j=1}^n \sum_{i=1}^{m} J_{ij}\nabla \beta_j * I = \sum_{i=1}^m \nabla y_i I$$ where $I$ represents the identity matrix. Then we have the final form of the equation:
$$\sum_{j=1}^n \sum_{i=1}^{m} J_{ij}\nabla \beta_j = \sum_{i=1}^m \nabla y_i$$
